In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
df = pd.read_json('http://localhost:5000/analyzed/whole-json', dtype={'name': 'str'})
df['geometry'] = df.apply(lambda r: Point(r.lon, r.lat), axis=1)
df.drop(['lat', 'lon'], axis=1, inplace=True)
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs=4326)
# gdf
gdf.describe()

In [ ]:
import folium
import branca

map = folium.Map(location=[35, 139], zoom_start=7)
fg = folium.FeatureGroup(name="Pika")
for n, row in df[[u'geometry', u'name', u'height']].iterrows():
    p, n, h = row[u'geometry'], row[u'name'], row[u'height']
    html = f'{n}<img src="http://localhost:5000/analyzed/image/{n}.png" width="400" />'
    iframe = branca.element.IFrame(html=html, width=400, height=350)
    marker = folium.Marker(location=(p.y, p.x), popup=folium.Popup(iframe, max_width=400))
    fg.add_child(marker)
map.add_child(fg)
map

In [ ]:
from IPython.display import display, HTML
from ipywidgets import IntRangeSlider, interactive

pitcher = IntRangeSlider(value=[-90, 0], min=-90, max=10, step=1)
adf = gdf

def on_value_change(pitch_range):
    p_min, p_max = pitch_range
    global adf
    adf = gdf[(p_min<=gdf.pitch)&(gdf.pitch<p_max)]
    urls = adf.name.apply(lambda n: f'http://localhost:5000/analyzed/image/{n}.png').to_list()
    imgs = [f"<img style='width: 300px; margin: 0px; float: left; border: 1px solid black;' src='{url}' />" for url in urls]
    display(HTML(''.join(imgs)))

interactive(on_value_change, pitch_range=pitcher)


In [ ]:
import json
import matplotlib.pyplot as plt

classes = json.load(open('cityscapes_labels.json'))['classes'][:-1]
cols = 3
bdf = adf.reset_index(drop=True)
fig, axes = plt.subplots(len(bdf)//cols+1, cols)
for i in bdf.index:
    axes[i//cols, i%cols].bar(classes, bdf.iloc[i].histogram)